In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import math

# Updated ET function using the new approach
def GetSatVapPres(temp):
    # Function to calculate saturation vapor pressure (kPa)
    return 0.611 * math.exp((17.27 * temp) / (temp + 237.3))

def ET(Ta, RH, SW, LW, PPFD, LAI, CropCover, Pressure, Area, k_T=0.58, epsilon_crop=0.98, sigma=5.67e-8, Mw=18.015, R=8.314, max_iter=100, tol=1e-3):
    ra = 120  # s/m
    rs = 60 * (1500 + PPFD) / (200 + PPFD)  # s/m
    hfg = 2.501e6  # J/kg

    # Initial guess for Ts
    Ts = Ta

    for _ in range(max_iter):
        # Calculate canopy cover
        tau_r = math.exp(-k_T * LAI) # Intercepted shortwave radiation
        rho_r = 0.27 # Reflectance of the canopy leaves
        Rsw = SW * (1 - tau_r) * (1 - rho_r) # Total absorbed radaition by the canopy

        # Saturation vapor pressures
        ea = GetSatVapPres(Ta) * RH / 100
        es = GetSatVapPres(Ts)
        Xa = ((ea * 1000) * Mw) / (R * (Ta + 273.15))
        Xs = ((es * 1000) * Mw) / (R * (Ts + 273.15))

        # Air properties
        rho_DA = CP.PropsSI('D', 'T', Ta + 273.15, 'P', Pressure * 1e3, 'Air')
        cp_DA = CP.PropsSI('C', 'T', Ta + 273.15, 'P', Pressure * 1e3, 'Air')

        # Sensible and latent heat fluxes
        qcrop_sens = (2 * rho_DA * cp_DA * LAI * (Ts - Ta)) / ra
        qcrop_lat = (LAI * ((Xs - Xa) / 1e3) * hfg) / (ra + rs)

        # Longwave radiation
        Rlw = CropCover * (LW - (sigma * epsilon_crop * (Ts + 273.15)**4))

        # Energy balance equation
        Rnet = Rsw + Rlw
        energy_balance = Rnet - qcrop_sens - qcrop_lat

        # Derivative of energy balance with respect to Ts (f'(Ts))
        dRlw_dTs = -4 * sigma * epsilon_crop * CropCover * (Ts + 273.15)**3
        dqcrop_sens_dTs = (2 * rho_DA * cp_DA * LAI) / ra
        dqcrop_lat_dTs = -(LAI * hfg * Mw * es * (17.27 * 237.3) / (R * (Ts + 273.15)**2 * ((Ts + 237.3)**2))) / (ra + rs)

        derivative = dRlw_dTs - dqcrop_sens_dTs - dqcrop_lat_dTs

        # Update Ts using Newton-Raphson
        Ts_new = Ts - energy_balance / derivative

        # Check for convergence
        if abs(Ts_new - Ts) < tol:
            Ts = Ts_new
            break

        Ts = Ts_new

    if _ == max_iter - 1 and abs(energy_balance) > tol:
        print(f"Warning: Newton-Raphson did not converge.")

    # Final results
    Q_lat = qcrop_lat  # Latent heat flux [W/m^2]
    Q_sens = qcrop_sens  # Sensible heat flux [W/m^2]
    M_lat = Q_lat / hfg  # Mass flux [kg/s/m^2]
    M_lat_tot = M_lat * Area  # Total mass flux [kg/s]

    return M_lat_tot, Ts, Rnet, Q_lat, Q_sens

